# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
import sentencepiece  # required by MarianTokenizer
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

/opt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get data from `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")

    # Standardize column names to [Title, Year, Synopsis, Review]
    df_eng.columns = ["Title", "Year", "Synopsis", "Review"]
    df_fr.columns = ["Title", "Year", "Synopsis", "Review"]
    df_sp.columns = ["Title", "Year", "Synopsis", "Review"]

    # Add Original Language column
    df_eng["Original Language"] = "en"
    df_fr["Original Language"] = "fr"
    df_sp["Original Language"] = "sp"

    # Concatenate all dataframes
    df = pd.concat([df_eng, df_fr, df_sp], ignore_index=True)
    return df

df = preprocess_data()

In [3]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",en
28,Torrente: El brazo tonto de la ley,1998,"En esta comedia española, un policía corrupto ...","""Torrente es una película vulgar y ofensiva qu...",sp
20,Roma,2018,Cleo (Yalitza Aparicio) es una joven empleada ...,"""Roma es una película hermosa y conmovedora qu...",sp
15,Le Dîner de Cons,1998,Le film suit l'histoire d'un groupe d'amis ric...,"""Je n'ai pas aimé ce film du tout. Le concept ...",fr
24,Amores perros,2000,Tres historias se entrelazan en esta película ...,"""Amores perros es una película intensa y conmo...",sp
11,Intouchables,2011,Ce film raconte l'histoire de l'amitié improba...,"""Intouchables est un film incroyablement touch...",fr
21,La Casa de Papel,(2017-2021),Esta serie de televisión española sigue a un g...,"""La Casa de Papel es una serie emocionante y a...",sp
27,El Bar,2017,Un grupo de personas quedan atrapadas en un ba...,"""El Bar es una película ridícula y sin sentido...",sp
22,Y tu mamá también,2001,Dos amigos adolescentes (Gael García Bernal y ...,"""Y tu mamá también es una película que se qued...",sp
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",en


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [5]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [6]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_mask = df["Original Language"] == "fr"
fr_reviews = df.loc[fr_mask, "Review"]
fr_reviews_en = fr_reviews.apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer))

# filter synopsis in French and translate to English
fr_synopsis = df.loc[fr_mask, "Synopsis"]
fr_synopsis_en = fr_synopsis.apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer))

# filter reviews in Spanish and translate to English
es_mask = df["Original Language"] == "sp"
es_reviews = df.loc[es_mask, "Review"]
es_reviews_en = es_reviews.apply(lambda x: translate(x, es_en_model, es_en_tokenizer))

# filter synopsis in Spanish and translate to English
es_synopsis = df.loc[es_mask, "Synopsis"]
es_synopsis_en = es_synopsis.apply(lambda x: translate(x, es_en_model, es_en_tokenizer))

# update dataframe with translated text
df.loc[fr_mask, "Review"] = fr_reviews_en.values
df.loc[fr_mask, "Synopsis"] = fr_synopsis_en.values
df.loc[es_mask, "Review"] = es_reviews_en.values
df.loc[es_mask, "Synopsis"] = es_synopsis_en.values

/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [7]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
22,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",sp
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",en
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",sp
16,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,"""I can't believe I've wasted time watching thi...",fr
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",sp
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",sp
10,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with...",fr
27,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",sp
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",fr


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [8]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    result = classifier(text[:512])[0]  # truncate to model max length
    label = result["label"]
    return "Positive" if label == "POSITIVE" else "Negative"

In [9]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df["Review Sentiment"] = df["Review"].apply(lambda x: analyze_sentiment(x, sentiment_classifier))

In [10]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Review Sentiment
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",fr,Negative
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",en,Negative
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",sp,Negative
14,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ...",fr,Positive
10,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with...",fr,Positive
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en,Positive
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",fr,Positive
11,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",fr,Positive
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",fr,Positive
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",en,Negative


In [12]:
# export the results to a .csv file
df.to_csv("reviews_with_sentiment.csv", index=False)